In [51]:
import urllib
import requests
from datetime import datetime
import json
import pprint

URL = 'https://st-dev-data-api.azurewebsites.net'
invertersURL = urllib.parse.urljoin(URL, '/api/v0.1/buildings/energyville1/pv/roof/inverters/')
powerURL = urllib.parse.urljoin(URL, '/api/v0.1/buildings/energyville1/pv/roof/power/')
user = 'eric_massip'
pswd = 'ssCpoDC3uHF4ssHJtvMf'

DATE_FORMAT_STR = "%Y-%m-%d"

start = datetime(2017, 6, 1)
end = datetime(2017, 6, 2)


def get_inverter_ids():
    inverter_ids_json_response = json.loads(requests.get(invertersURL, auth=(user, pswd)).text)['data']
    inverter_ids = []
    for i in range(len(inverter_ids_json_response)):
        inverter_ids.append(inverter_ids_json_response[i]['InverterId'])
    return inverter_ids


def get_params(inverter_id):
    return {'start': start.strftime(DATE_FORMAT_STR),
            'end': end.strftime(DATE_FORMAT_STR),
            'time_zone': 'Central European Standard Time',
            'inverter_ids': inverter_id}


def get_power_consumption(inverter_id):
    return json.loads(requests.get(powerURL, params=get_params(inverter_id), auth=(user, pswd)).text)['data']


def get_all_power_consumptions():
    inverter_ids = get_inverter_ids()
    return [get_power_consumption(inverter_id) for inverter_id in inverter_ids]


all_power_consumptions = get_all_power_consumptions()

In [113]:
from datetime import timedelta

timedelta_5 = timedelta(minutes=5)

caca = start + timedelta_5

dates = [start]
for i in range(1, 288):
    dates.append(dates[i - 1] + timedelta_5)

In [109]:
total_power_consumption = []

for date in dates:
    power_sum = 0
    for inverter_consumption in all_power_consumptions:
        for i in range(len(inverter_consumption)):
            log_generation_power = inverter_consumption[i]['GenerationPower']
            if log_generation_power != 0:
                log_hour = datetime.fromisoformat(inverter_consumption[i]['DateTimeMeasurement']).hour
                log_minute = datetime.fromisoformat(inverter_consumption[i]['DateTimeMeasurement']).minute
                if log_hour == date.hour and log_minute == date.minute:
                    power_sum += log_generation_power
    total_power_consumption.append({'date': date.strftime("%Y-%m-%d %H:%M:%S"), 'power_sum': power_sum})

2017-06-01 00:00:00+02:00
2017-06-01 00:00:00+02:00
True


In [110]:
total_power_consumption

[{'date': '2017-06-01 00:00:00', 'power_sum': 0},
 {'date': '2017-06-01 00:05:00', 'power_sum': 0},
 {'date': '2017-06-01 00:10:00', 'power_sum': 0},
 {'date': '2017-06-01 00:15:00', 'power_sum': 0},
 {'date': '2017-06-01 00:20:00', 'power_sum': 0},
 {'date': '2017-06-01 00:25:00', 'power_sum': 0},
 {'date': '2017-06-01 00:30:00', 'power_sum': 0},
 {'date': '2017-06-01 00:35:00', 'power_sum': 0},
 {'date': '2017-06-01 00:40:00', 'power_sum': 0},
 {'date': '2017-06-01 00:45:00', 'power_sum': 0},
 {'date': '2017-06-01 00:50:00', 'power_sum': 0},
 {'date': '2017-06-01 00:55:00', 'power_sum': 0},
 {'date': '2017-06-01 01:00:00', 'power_sum': 0},
 {'date': '2017-06-01 01:05:00', 'power_sum': 0},
 {'date': '2017-06-01 01:10:00', 'power_sum': 0},
 {'date': '2017-06-01 01:15:00', 'power_sum': 0},
 {'date': '2017-06-01 01:20:00', 'power_sum': 0},
 {'date': '2017-06-01 01:25:00', 'power_sum': 0},
 {'date': '2017-06-01 01:30:00', 'power_sum': 0},
 {'date': '2017-06-01 01:35:00', 'power_sum': 0},
